In [2]:
import pandas as pd
import datetime as dt

# Data Request for May 21, 2025
# How many clients transitioned into housing with any organization during QX XXXX after being served by SALT from XXXX - XXXX?
# See Data Request Guide in Google Drive for more info and how to pull from Data Explorer

serviceBeginDate = dt.datetime(2020, 12, 31)
serviceEndDate = dt.datetime(2025, 12, 31)

successBeginDate = dt.datetime(2025, 10, 1)
successEndDate = dt.datetime(2025, 12, 31)

# Format dates for output
output1 = successBeginDate.strftime('%B %d %Y')
output2 = successEndDate.strftime('%B %d %Y')
output3 = serviceBeginDate.strftime('%B %d %Y')
output4 = serviceEndDate.strftime('%B %d %Y')

# Read data
dfServices = pd.read_csv('Services.csv')
dfSuccess = pd.read_csv('Successes.csv')

dfServices['Service Date'] = pd.to_datetime(dfServices['Service Date'])
dfSuccess['Exit Date'] = pd.to_datetime(dfSuccess['Exit Date'])

dfServicesOG = dfServices.copy()

dfServices['org'] = dfServices['Provider'].fillna(dfServices['Associated Program'])

dfServices = dfServices.dropna(subset=['org'])
dfServices = dfServices[dfServices['org'].str.contains('SALT')]

dfServicesInTimeframe = dfServices[(dfServices['Service Date'] >= serviceBeginDate) & (dfServices['Service Date'] <= serviceEndDate)]

dfSuccessInTimeframe = dfSuccess[(dfSuccess['Exit Date'] >= successBeginDate) & (dfSuccess['Exit Date'] <= successEndDate)]

# Define a successful Housing Exit
Housing = [
    'Permanent housing (other than RRH) for formerly homeless persons',
    'Owned by client, no ongoing housing subsidy',
    'Rental by client, no ongoing housing subsidy',
    'Rental by client, VASH housing Subsidy',
    'Rental by client, other ongoing housing subsidy',
    'Rental by client in a public housing unit',
    'Rental by client with RRH or equivalent subsidy',
    'Long-term care facility or nursing home',
    'Residential project or halfway house with no homeless criteria',
    'Owned by client, with ongoing housing subsidy',
    'Rental by client, with HCV voucher (tenant or project based)',
    'Rental by client, with GPD TIP housing subsidy'
    ]

# Start output
print(f"-------- Successful Housing Transitions Report --------")
print('\n')

# Total number of unduplicated clients served
total_clients_served = dfServicesInTimeframe['Client ID'].nunique()
print(f"Total of unduplicated clients served from {output3} to {output4}: {total_clients_served}")
print('\n')

# After receiving a SALT service, how many transitioned into housing?
dfServicesInTimeframe = dfServicesInTimeframe.sort_values('Service Date').drop_duplicates('Client ID', keep='first')

dfIndirectSucccess = dfSuccessInTimeframe[dfSuccessInTimeframe['Exit Destination'].isin(Housing)]
dfIndirectSucccess = dfIndirectSucccess.sort_values('Exit Date').drop_duplicates('Client ID', keep='last')

dfIndirectSucccess = pd.merge(dfServicesInTimeframe, dfIndirectSucccess, on='Client ID', how='inner')

dfIndirectSucccess['Exit Date'] = pd.to_datetime(dfIndirectSucccess['Exit Date'])
dfIndirectSucccess['Service Date'] = pd.to_datetime(dfIndirectSucccess['Service Date'])

dfIndirectSucccess = dfIndirectSucccess[dfIndirectSucccess['Exit Date'] > dfIndirectSucccess['Service Date']]

dfIndirectSucccess.to_excel('exitids.xlsx', index=False)

# uncomment below code if you want to specify a gap of time between a service and exit i.e. a year
# dfIndirectSucccess = dfIndirectSucccess[dfIndirectSucccess['Exit Date'] - dfIndirectSucccess['Service Date'] <= pd.Timedelta(365, 'D')]

print(len(dfIndirectSucccess), 'people transitioned into housing from {0} to {1} after receiving a SALT service from {2} to {3}'.format(output1, output2, output3, output4) )


/var/folders/nm/sgb0qrh5259964c0_5jt6zfh0000gn/T/ipykernel_27155/2470337983.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfServices['Service Date'] = pd.to_datetime(dfServices['Service Date'])
/var/folders/nm/sgb0qrh5259964c0_5jt6zfh0000gn/T/ipykernel_27155/2470337983.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfSuccess['Exit Date'] = pd.to_datetime(dfSuccess['Exit Date'])


-------- Successful Housing Transitions Report --------


Total of unduplicated clients served from December 31 2020 to December 31 2025: 5181


50 people transitioned into housing from October 01 2025 to December 31 2025 after receiving a SALT service from December 31 2020 to December 31 2025
